In [154]:
import requests

import matplotlib.pyplot as plt
import numpy as np
import cv2
import math

import geopandas as gpd
import geojson
from shapely.geometry import Point, LineString
import flexpolyline


In [155]:
def image_to_corners(image):
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)

    corners = cv2.goodFeaturesToTrack(gray, 100, 0.0001, 10)
    corners = np.int0(corners)
    
    return corners

corners_1 = image_to_corners('test_image_1.png')
corners_2 = image_to_corners('test_image_2.jpg')
corners_3 = image_to_corners('test_image_3.jpg')

In [156]:
squeezed_1 = corners_1.squeeze().tolist()
squeezed_2 = corners_2.squeeze().tolist()
squeezed_3 = corners_3.squeeze().tolist()

In [157]:
def distance(P1, P2):
    """
    To compute the distance between 2 points defined by
    P1 = (x1, y1) and P2 = (x2, y2)
    """
    return ((P1[0] - P2[0])**2 + (P1[1] - P2[1])**2)**0.5


def optimized_path(coords, start=None):
    """
    To find the nearest point to a point.
    Coords format: [ [x1, y1], [x2, y2], ... ]
    """
    if start is None:
        start = coords[0]
    pass_by = coords
    path = [start]
    pass_by.remove(start)
    while pass_by:
        nearest = min(pass_by, key=lambda x: distance(path[-1], x))
        path.append(nearest)
        pass_by.remove(nearest)
    return path

In [158]:
path_1 = optimized_path(squeezed_1)
path_2 = optimized_path(squeezed_2)
path_3 = optimized_path(squeezed_3)

In [159]:
x_1 = np.array([i[0] for i in path_1])
y_1 = np.array([i[1] for i in path_1])
matrix_1 = np.array([x_1, -y_1])

x_2 = np.array([i[0] for i in path_2])
y_2 = np.array([i[1] for i in path_2])

matrix_2 = np.array([x_2, -y_2])

x_3 = np.array([i[0] for i in path_3])
y_3 = np.array([i[1] for i in path_3])

matrix_3 = np.array([x_3, -y_3])

In [160]:
meters = 10

# Координаты в системе координат EPSG:3857

x_original_point = float(input("Введите широту в формате EPSG 3857: "))
y_original_point = float(input("Введите долготу в формате EPSG 3857: "))

mx_1 = x_1 * meters + x_original_point
my_1 = y_1 * -meters + y_original_point

mxy_1 = list(zip(mx_1, my_1))

picture_df_1 = gpd.GeoDataFrame(
    {'id': range(0, len(mxy_1))},
    crs="EPSG:3857",
    geometry=[Point(resu) for resu in mxy_1]
)

picture_df_1['geometry'] = picture_df_1['geometry'].to_crs(epsg=4236)

picture_df_1.to_file("img1.geojson", driver="GeoJSON", encoding="")

mx_2 = x_2 * meters + x_original_point
my_2 = y_2 * -meters + y_original_point

mxy_2 = list(zip(mx_2, my_2))

picture_df_2 = gpd.GeoDataFrame(
    {'id': range(0, len(mxy_2))},
    crs="EPSG:3857",
    geometry=[Point(resu) for resu in mxy_2]
)

picture_df_2['geometry'] = picture_df_2['geometry'].to_crs(epsg=4236)

picture_df_2.to_file("img2.geojson", driver="GeoJSON", encoding="")

mx_3 = x_3 * meters + x_original_point
my_3 = y_3 * -meters + y_original_point

mxy_3 = list(zip(mx_3, my_3))

picture_df_3 = gpd.GeoDataFrame(
    {'id': range(0, len(mxy_3))},
    crs="EPSG:3857",
    geometry=[Point(resu) for resu in mxy_3]
)

picture_df_3['geometry'] = picture_df_3['geometry'].to_crs(epsg=4236)

picture_df_3.to_file("img3.geojson", driver="GeoJSON", encoding="")

Введите широту в формате EPSG 3857: 6819755.61
Введите долготу в формате EPSG 3857: 12275623.63


In [170]:
API_KEY = 'dSOFgNcgQXhsQUxf6xiZ817_iaS6VMlANfTnKVHCO0Y'
SERVICE = 'https://router.hereapi.com/v8/routes?apiKey=dSOFgNcgQXhsQUxf6xiZ817_iaS6VMlANfTnKVHCO0Y&transportMode=pedestrian&return=polyline'
file = open('img1.geojson')
data_1 = geojson.load(file).copy()

file.close()

file = open('img2.geojson')
data_2 = geojson.load(file).copy()

file.close()

file = open('img3.geojson')
data_3 = geojson.load(file).copy()

file.close()

In [171]:
coords_list_1 = [feature['geometry']['coordinates'] for feature in data_1['features']]

start_point_1 = coords_list_1[0]
destination_point_1 = coords_list_1[len(coords_list_1) - 1]

coords_list_1.remove(start_point_1)
coords_list_1.remove(destination_point_1)

coords_list_2 = [feature['geometry']['coordinates'] for feature in data_2['features']]

start_point_2 = coords_list_2[0]
destination_point_2 = coords_list_2[len(coords_list_2) - 1]

coords_list_2.remove(start_point_2)
coords_list_2.remove(destination_point_2)

coords_list_3 = [feature['geometry']['coordinates'] for feature in data_3['features']]

start_point_3 = coords_list_3[0]
destination_point_3 = coords_list_3[len(coords_list_3) - 1]

coords_list_3.remove(start_point_3)
coords_list_3.remove(destination_point_3)

In [172]:
origin_1 = f'&origin={start_point_1[1]},{start_point_1[0]}'
destination_1 = f'&destination={destination_point_1[1]},{destination_point_1[0]}&'
waypoints_1 = '&'.join([f'via={coords[1]},{coords[0]}' for coords in coords_list_1])

origin_2 = f'&origin={start_point_2[1]},{start_point_2[0]}'
destination_2 = f'&destination={destination_point_2[1]},{destination_point_2[0]}&'
waypoints_2 = '&'.join([f'via={coords[1]},{coords[0]}' for coords in coords_list_2])

origin_3 = f'&origin={start_point_3[1]},{start_point_3[0]}'
destination_3 = f'&destination={destination_point_3[1]},{destination_point_3[0]}&'
waypoints_3 = '&'.join([f'via={coords[1]},{coords[0]}' for coords in coords_list_3])

In [178]:
routes_1 = requests.get(SERVICE + origin_1 + destination_1 + waypoints_1).json()
routes_2 = requests.get(SERVICE + origin_2 + destination_2 + waypoints_2).json()
routes_3 = requests.get(SERVICE + origin_3 + destination_3 + waypoints_3).json()
#routes.url

In [177]:
def decode(section):
    line = flexpolyline.decode(section['polyline'])
    line = [(coord[1], coord[0]) for coord in line]
    return LineString(line)

geometry_1 = [ decode(section) for section in routes_1['routes'][0]['sections'] ]
geometry_2 = [ decode(section) for section in routes_2['routes'][0]['sections'] ]
geometry_3 = [ decode(section) for section in routes_3['routes'][0]['sections'] ]

IndexError: list index out of range

In [176]:
route_df_1 = gpd.GeoDataFrame(geometry=geometry_1)
route_df_2 = gpd.GeoDataFrame(geometry=geometry_2)
route_df_3 = gpd.GeoDataFrame(geometry=geometry_3)

route_df_1.to_file("route1.geojson", driver="GeoJSON", encoding="utf-8")
route_df_2.to_file("route2.geojson", driver="GeoJSON", encoding="utf-8")
route_df_3.to_file("route3.geojson", driver="GeoJSON", encoding="utf-8")

NameError: name 'geometry_1' is not defined

In [136]:
from pyproj import Proj, transform
x_original_4326,x_original_4326 = transform("epsg:3857","epsg:4326",x_original_point,y_original_point)


import folium

m1 = folium.Map(
    location = [x_original_4326, y_original_4326],
    zoom_start = 15,
    tiles='https://1.base.maps.ls.hereapi.com/maptile/2.1/maptile/newest/reduced.day/{z}/{x}/{y}/256/png?lg=RU&apiKey=dSOFgNcgQXhsQUxf6xiZ817_iaS6VMlANfTnKVHCO0Y',
    attr='HERE'
)

folium.GeoJson('img1.geojson', name="geojson").add_to(m1)
folium.GeoJson('route1.geojson', name="geojson").add_to(m1)

m1.save('map1.html')

m2 = folium.Map(
    location = [x_original_4326, y_original_4326],
    zoom_start = 15,
    tiles='https://1.base.maps.ls.hereapi.com/maptile/2.1/maptile/newest/reduced.day/{z}/{x}/{y}/256/png?lg=RU&apiKey=dSOFgNcgQXhsQUxf6xiZ817_iaS6VMlANfTnKVHCO0Y',
    attr='HERE'
)

folium.GeoJson('img2.geojson', name="geojson").add_to(m2)
folium.GeoJson('route2.geojson', name="geojson").add_to(m2)

m2.save('map2.html')

m3 = folium.Map(
    location = [x_original_4326, y_original_4326],
    zoom_start = 15,
    tiles='https://1.base.maps.ls.hereapi.com/maptile/2.1/maptile/newest/reduced.day/{z}/{x}/{y}/256/png?lg=RU&apiKey=dSOFgNcgQXhsQUxf6xiZ817_iaS6VMlANfTnKVHCO0Y',
    attr='HERE'
)

folium.GeoJson('img3.geojson', name="geojson").add_to(m3)
folium.GeoJson('route3.geojson', name="geojson").add_to(m3)

m3.save('map3.html')


In [137]:
m1
m2
m3